## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.65,115.33,57.99,76,0,5.99,clear sky
1,1,Ushuaia,AR,-54.80,-68.30,46.40,58,75,25.28,broken clouds
2,2,Rikitea,PF,-23.12,-134.97,74.50,68,29,11.72,scattered clouds
3,3,Hilo,US,19.73,-155.09,84.20,58,40,10.29,scattered clouds
4,4,Punta Arenas,CL,-53.15,-70.92,53.60,40,75,16.11,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.12,-134.97,74.50,68,29,11.72,scattered clouds
3,3,Hilo,US,19.73,-155.09,84.20,58,40,10.29,scattered clouds
6,6,Winslow,US,35.02,-110.70,73.40,18,1,5.55,clear sky
9,9,Sao Filipe,CV,14.90,-24.50,76.95,76,10,5.50,clear sky
20,20,Avarua,CK,-21.21,-159.78,75.20,83,83,5.82,broken clouds
23,23,Maningrida,AU,-12.07,134.27,82.17,69,0,4.00,clear sky
24,24,Mecca,SA,21.43,39.83,75.67,60,0,4.29,clear sky
25,25,Taylorville,US,39.55,-89.29,71.01,37,1,12.75,clear sky
28,28,Georgetown,MY,5.41,100.34,78.80,88,20,2.24,thunderstorm
34,34,Bambous Virieux,MU,-20.34,57.76,73.40,78,40,6.93,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.12,-134.97,74.50,68,29,11.72,scattered clouds
3,3,Hilo,US,19.73,-155.09,84.20,58,40,10.29,scattered clouds
6,6,Winslow,US,35.02,-110.70,73.40,18,1,5.55,clear sky
9,9,Sao Filipe,CV,14.90,-24.50,76.95,76,10,5.50,clear sky
20,20,Avarua,CK,-21.21,-159.78,75.20,83,83,5.82,broken clouds


In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,74.50,scattered clouds,-23.12,-134.97,
3,Hilo,US,84.20,scattered clouds,19.73,-155.09,
6,Winslow,US,73.40,clear sky,35.02,-110.70,
9,Sao Filipe,CV,76.95,clear sky,14.90,-24.50,
20,Avarua,CK,75.20,broken clouds,-21.21,-159.78,
23,Maningrida,AU,82.17,clear sky,-12.07,134.27,
24,Mecca,SA,75.67,clear sky,21.43,39.83,
25,Taylorville,US,71.01,clear sky,39.55,-89.29,
28,Georgetown,MY,78.80,thunderstorm,5.41,100.34,
34,Bambous Virieux,MU,73.40,scattered clouds,-20.34,57.76,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {"radius": 5000, "type": "lodging", "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
     # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [25]:
import numpy as np

In [32]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,74.50,scattered clouds,-23.12,-134.97,Pension Maro'i
3,Hilo,US,84.20,scattered clouds,19.73,-155.09,Hilo Hawaiian Hotel
6,Winslow,US,73.40,clear sky,35.02,-110.70,Best Western Plus Winslow Inn
9,Sao Filipe,CV,76.95,clear sky,14.90,-24.50,Tortuga B&B
20,Avarua,CK,75.20,broken clouds,-21.21,-159.78,Paradise Inn
...,...,...,...,...,...,...,...
663,Maiduguri,NG,77.34,scattered clouds,11.85,13.16,College of Nursing & Midwifery Female's Hostel
664,Seminole,US,78.80,few clouds,28.71,-81.21,Ono Hot Dog Cart and Catering
670,Zabid,YE,79.74,clear sky,14.20,43.32,دار الضيافة
672,Sibolga,ID,76.86,overcast clouds,1.74,98.78,Hotel Syariah CN Darussalam


In [33]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [40]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [41]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))